# Challenge 

## Vigenère cipher

In [1]:
import numpy as np
import collections
import operator
import pandas as pd

PLAIN_ALPHABET = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [2]:
def load(fname):
    f = open(fname,'r')
    #data = []
    contents = f.read()
    file_as_list = contents.splitlines(False) 
    
    data = [line for line in file_as_list if line != ""]
        
    f.close()
    
    return data

In [3]:
def shift_alpha(shift):
    alpha_list = list(PLAIN_ALPHABET)
    return ''.join(alpha_list[shift:] + alpha_list[:shift])

In [89]:
def count_frequencies(cipher, substring_size = 1, positions = "all"):
    
    cipher = ' '.join(cipher).replace(" ", "")
    count_dict = {}
    
    if positions == "all":
        positions_to_count = range(len(cipher))
    else:
        positions_to_count = np.arange(positions, len(cipher), 5)
    
    for position in positions_to_count:
        try:
            count_dict[cipher[position:position+substring_size]] = count_dict[
                cipher[position:position+substring_size]] + 1
            
        except KeyError:
            count_dict[cipher[position:position+substring_size]] = 1
    
    dataframe = pd.DataFrame(sorted(count_dict.items(), key=operator.itemgetter(1)),
                             columns=["letter", "count"])
    dataframe["percentage"] = dataframe["count"]/len(cipher)
    return dataframe.sort_values(by="percentage", ascending=False)

In [90]:
def find_all(string,substring):

    length = len(substring)
    c=0
    indexes = []
    while c < len(string):
        if string[c:c+length] == substring:
            indexes.append(c)
        c=c+1
    return indexes

def get_digram_ocurrences(digrams,cipher):
    
    cipher = ' '.join(cipher).replace(" ", "")
    
    for digram in digrams:
        positions = find_all(cipher, digram)
        print(np.diff(positions))
        
    return

In [314]:
def decrypt(cipher, alpha, keyword):
    cipher = ' '.join(cipher).replace(" ", "")
    
    decrypted_msg = []    
    shifts = [PLAIN_ALPHABET.index(letter) for letter in keyword]             
    alpha_cipher = [shift_alpha(i) for i in shifts]
   
    
    for position in np.arange(0, len(cipher), 5):
        for i in range(5):
            try:
                decrypted_msg.append(alpha[alpha_cipher[i].index(cipher[position+i])])
            except IndexError:
                break
                
    print (''.join(decrypted_msg).lower())
    #return decrypted_msg

In [315]:
# Load file
PATH = "./cipher.txt"

CIPHER = load(PATH)

CIPHER

['K Q O W E F V J P U J U U N U K G L M E K J I N M W U X F Q M K J B',
 'G W R L F N F G H U D W U U M B S V L P S N C M U E K Q C T E S W R',
 'E E K O Y S S I W C T U A X Y O T A P X P L W P N T C G O J B G F Q',
 'H T D W X I Z A Y G F F N S X C S E Y N C T S S P N T U J N Y T G G',
 'W Z G R W U U N E J U U Q E A P Y M E K Q H U I D U X F P G U Y T S',
 'M T F F S H N U O C Z G M R U W E Y T R G K M E E D C T V R E C F B',
 'D J Q C U S W V B P N L G O Y L S K M T E F V J J T W W M F M W P N',
 'M E M T M H R S P X F S S K F F S T N U O C Z G M D O E O Y E E K C',
 'P J R G P M U R S K H F R S E I U E V G O Y C W X I Z A Y G O S A A',
 'N Y D O E O Y J L W U N H A M E B F E L X Y V L W N O J N S I O F R',
 'W U C C E S W K V I D G M U C G O C R U W G N M A A F F V N S I U D',
 'E K Q H C E U C P F C M P V S U D G A V E M N Y M A M V L F M A O Y',
 'F N T Q C U A F V F J N X K L N E I W C W O D C C U L W R I F T W G',
 'M U S W O V M A T N Y B U H T C O C W F Y T N M G Y T Q M K B 

First, let's try to find the word length checking the digrams.

In [316]:
digrams_frequencies = count_frequencies(CIPHER, substring_size = 2)

digrams_frequencies.head(10)

,letter,count,percentage
316,GM,9,0.014901
315,EK,8,0.013245
314,DG,6,0.009934
313,ME,6,0.009934
312,EE,6,0.009934
311,OY,6,0.009934
310,GO,6,0.009934
309,WU,6,0.009934
305,UU,5,0.008278
303,WG,5,0.008278


In [317]:
get_digram_ocurrences(digrams_frequencies["letter"][:3], CIPHER)

[80 90 90 75  5 40 10  5]
[ 40  10  85 115 105 160   5]
[ 40 125   5  55  10]


The common factor is 5, that's the key size :)

Once we had find the key size, let's do some frequency analysis: every fifth letter will be encrypted with the same letter of the key.

In [318]:
count_frequencies(CIPHER, substring_size = 1, positions = 0)

,letter,count,percentage
19,W,24,0.039735
18,S,13,0.021523
17,K,11,0.018212
16,D,10,0.016556
15,M,9,0.014901
14,J,7,0.011589
13,A,7,0.011589
12,G,6,0.009934
11,F,5,0.008278
10,Y,4,0.006623


In [319]:
count_frequencies(CIPHER, substring_size = 1, positions = 1)

,letter,count,percentage
16,G,26,0.043046
15,U,9,0.014901
14,N,9,0.014901
9,C,8,0.013245
13,W,8,0.013245
12,P,8,0.013245
11,O,8,0.013245
10,K,8,0.013245
8,V,7,0.011589
7,Q,7,0.011589


In [320]:
count_frequencies(CIPHER, substring_size = 1, positions = 2)

,letter,count,percentage
21,Y,19,0.031457
20,M,15,0.024834
19,C,11,0.018212
18,U,10,0.016556
17,O,8,0.013245
16,F,8,0.013245
15,N,7,0.011589
14,L,6,0.009934
13,I,6,0.009934
12,J,5,0.008278


In [321]:
count_frequencies(CIPHER, substring_size = 1, positions = 3)

,letter,count,percentage
20,F,17,0.028146
19,T,13,0.021523
18,U,12,0.019868
17,B,11,0.018212
16,V,7,0.011589
15,M,7,0.011589
14,O,6,0.009934
13,J,6,0.009934
12,E,6,0.009934
11,S,5,0.008278


In [322]:
count_frequencies(CIPHER, substring_size = 1, positions = 4)

,letter,count,percentage
17,E,27,0.044702
16,R,12,0.019868
15,N,10,0.016556
14,U,9,0.014901
13,S,9,0.014901
12,T,8,0.013245
11,A,8,0.013245
10,L,7,0.011589
9,C,7,0.011589
8,I,5,0.008278


So, based on a frequency analysis, letters "W, G, Y, F, E" should corresponds to "E" on plain alphabet.

Looking for the correspondences on the Vigenere Table, we identify the keyword as "SCUBA". 

No, we just need to decode...


In [323]:
decrypt(CIPHER, PLAIN_ALPHABET, "SCUBA")

souventpoursamuserleshommesdequipageprennentdesalbatrosvastesoiseauxdesmersquisuiventindolentscompagnonsdevoyagelenavireglissantsurlesgouffresamersapeinelesontilsdeposessurlesplanchesquecesroisdelazurmaladroitsethonteuxlaissentpiteusementleursgrandesailesblanchescommedesavironstraineracotedeuxcevoyageurailecommeilestgaucheetveuleluinagueresibeauquilestcomiqueetlaidlunagacesonbecavecunbrulegueulelautremimeenboitantlinfirmequivolaitlepoeteestsemblableauprincedesnueesquihantelatempeteetseritdelarcherbaudelaireexilesurlesolaumilieudeshueeslemotpouretagequatreesttrajansesailesdegeantlempechentdemarcher
